# Soving direct and inverse problems of a single stage refrigeration vapor-comression cycle
### Demonstration of the Thermal-Hydraulic Computation Framework

This notebook demonstrates the solution of a steady-state heat pump
network using the framework introduced in:

> M. Ganz (2025)  
> *Computational Modelling Framework for Solving Direct and Inverse Problems  
> in Steady-State Thermal-Hydraulic Network Calculations*

The example covers:
- automatic network topology construction
- tripartite graph formulation
- tearing-based execution order
- loop breaking in closed fluid cycles
- direct simulation
- inverse optimization


In [17]:
# import the framework
from thdin import *

## Problem Description

We consider a steady-state heat pump consisting of:

- Evaporator
- Compressor
- Condenser
- Expansion valve
- Internal heat exchanger (IHX)
- Receiver

The system includes:
- one closed refrigerant loop (R134a)
- two secondary incompressible loops (antifrogen)

Both **direct** (forward simulation) and **inverse** (parameter estimation /
design optimization) problems are solved using the same network formulation.


In [18]:
net = Network()

net.fluid_loops[1] = "R134a"
net.fluid_loops[2] = "INCOMP::AN[0.4]"
net.fluid_loops[3] = "INCOMP::AN[0.4]"


## Component Models

Each component is defined by:
- a set of ports
- a causal modeling type (pressure-based / mass-flow-based)
- a local component model

First one needs to generate the component instances of the network by defining the causal modeling type and ports.


In [19]:
# Defining component models of the network

# Evaporator
evap = MassFlowBasedComponent(
    label="Evap",
    port_specs={
        "in_af":  PortSpec("in"),
        "out_af": PortSpec("out"),
        "in_r": PortSpec("in"),
        "out_r":PortSpec("out"),
    },
)

# Compressor
comp = PressureBasedComponent(
    label="Comp",
    port_specs={
        "in":  PortSpec("in"),
        "out": PortSpec("out"),
    },
)

# Condenser
cond = MassFlowBasedComponent(
    label="Cond",
    port_specs={
        "in_r":  PortSpec("in"),
        "out_r": PortSpec("out"),
        "in_af": PortSpec("in"),
        "out_af":PortSpec("out"),
    },
)

# Expansion Valve
exp = PressureBasedComponent(
    label="Exp",
    port_specs={
        "in":  PortSpec("in"),
        "out": PortSpec("out"),
    },
)

# Receiver
rev = MassFlowBasedComponent(
    label="Rev",
    port_specs={
        "in":  PortSpec("in"),
        "out": PortSpec("out"),
    },
)

# Internal Heat Exchanger
ihx = MassFlowBasedComponent(
    label="IHX",
    port_specs={
        "in_hot":  PortSpec("in"),
        "out_hot": PortSpec("out"),
        "in_cold": PortSpec("in"),
        "out_cold":PortSpec("out"),
    },
)

# adds components to the network
net.add_component(comp)
net.add_component(cond)
net.add_component(evap)
net.add_component(exp)
net.add_component(ihx)
net.add_component(rev)


#### Component model definition

Once the components are generated and added to the network, the specific numerical models of each component can be defined of the form func(comp: Component) and then set.

In [20]:
# Evaporator model
def evap_model(evap):

    N = 50
    k = 1000

    A = evap.parameter['A'].value

    hi_h = evap.ports['in_af'].h.value
    pi_h = evap.ports['in_af'].p.value
    m_h = evap.ports['in_af'].m.value
    hot_fluid = evap.ports['in_af'].fluid

    hi_c = evap.ports['in_r'].h.value
    pi_c = evap.ports['in_r'].p.value
    m_c = evap.ports['in_r'].m.value
    cold_fluid = evap.ports['in_r'].fluid

    def res(x):
        h_h = np.zeros(N)
        h_c = np.zeros(N)
        h_c[0] = hi_c
        h_h[0] = x[0] * 1e5
        for i in range(N - 1):
            Th = PropsSI('T', 'H', h_h[i], 'P', pi_h, hot_fluid)
            Tc = PropsSI('T', 'H', h_c[i], 'P', pi_c, cold_fluid)
            h_h[i + 1] = h_h[i] + k * A / N * (Th - Tc) / m_h
            h_c[i + 1] = h_c[i] + k * A / N * (Th - Tc) / m_c
        return np.array([(h_h[-1] - hi_h) * 1e-5])

    Ti_c = PropsSI('T', 'H', hi_c, 'P', pi_c, cold_fluid)
    ho_h_init = PropsSI('H', 'P', pi_h, 'T', max(Ti_c, 257.0), hot_fluid) * 1e-5
    sol = scipy.optimize.root(res, np.array([ho_h_init]), method='hybr', tol=1e-5)

    if sol.success:
        ho_h = sol.x[0] * 1e5
        h_h = np.zeros(N)
        h_c = np.zeros(N)
        h_c[0] = hi_c
        h_h[0] = ho_h
        for i in range(N - 1):
            Th = PropsSI('T', 'H', h_h[i], 'P', pi_h, hot_fluid)
            Tc = PropsSI('T', 'H', h_c[i], 'P', pi_c, cold_fluid)
            h_h[i + 1] = h_h[i] + k * A / N * (Th - Tc) / m_h
            h_c[i + 1] = h_c[i] + k * A / N * (Th - Tc) / m_c
    else:
        raise Exception(sol.message)

    po_h = pi_h
    ho_h = h_h[0]
    mo_h = m_h

    po_c = pi_c
    ho_c = h_c[-1]
    mo_c = m_c

    evap.ports['out_af'].p.set_value(po_h)
    evap.ports['out_af'].h.set_value(ho_h)
    evap.ports['out_af'].m.set_value(mo_h)

    evap.ports['out_r'].p.set_value(po_c)
    evap.ports['out_r'].h.set_value(ho_c)
    evap.ports['out_r'].m.set_value(mo_c)

    evap.outputs["Q"].set_value(m_c * (ho_c - hi_c))

# Compressor model
def comp_model(comp):

    n = 4 # number of cylinders
    bore = 0.06 # bore [m]
    stroke = 0.042 # stroke [m]
    Vd = n * bore**2 * np.pi / 4 * stroke # displacement volume

    a = np.array([-5.31166292e-02, 1.21402922e-03, 8.81226071e-05, 1.03163725e+00])
    b = np.array([9.38116126e-03, -1.52858792e-03, -4.08026601e-03, 6.31332600e-04, 6.77625196e-01])

    RPM = comp.parameter['RPM'].value
    hi = comp.ports['in'].h.value
    pi = comp.ports['in'].p.value
    po = comp.ports['out'].p.value
    fluid = comp.ports['in'].fluid

    eta_v = a[0] * (po / pi) + a[1] * (po / pi) ** 2 + a[2] * (RPM / 60) + a[3]
    eta_is = b[0] * (po / pi) + b[1] * (po / pi) ** 2 + b[2] * (RPM / 60) + b[3] * (po / pi) * (RPM / 60) + b[4]

    s = PropsSI('S', 'P', pi, 'H', hi, fluid)
    h_is = PropsSI('H', 'P', po, 'S', s, fluid)
    rho = PropsSI('D', 'P', pi, 'H', hi, fluid)
    ho = hi + (h_is - hi) / eta_is
    mi = mo = RPM / 60 * Vd * eta_v * rho

    comp.ports['in'].m.set_value(mi)
    comp.ports['out'].h.set_value(ho)
    comp.ports['out'].m.set_value(mo)

# Condenser model
def cond_model(cond):

    N = 50
    k = 1000

    A = cond.parameter['A'].value

    hi_h = cond.ports["in_r"].h.value
    pi_h = cond.ports["in_r"].p.value
    m_h = cond.ports["in_r"].m.value
    hot_fluid = cond.ports["in_r"].fluid

    hi_c = cond.ports["in_af"].h.value
    pi_c = cond.ports["in_af"].p.value
    m_c = cond.ports["in_af"].m.value
    cold_fluid = cond.ports["in_af"].fluid

    def res(x):
        h_h = np.zeros(N)
        h_c = np.zeros(N)
        h_h[0] = hi_h
        h_c[0] = x[0] * 1e5
        for i in range(N - 1):
            Th = PropsSI('T', 'H', h_h[i], 'P', pi_h, hot_fluid)
            Tc = PropsSI('T', 'H', h_c[i], 'P', pi_c, cold_fluid)
            h_h[i + 1] = h_h[i] - k * A / N * (Th - Tc) / m_h
            h_c[i + 1] = h_c[i] - k * A / N * (Th - Tc) / m_c
        return np.array([(h_c[-1] - hi_c) * 1e-5])

    Ti_h = PropsSI('T', 'H', hi_h, 'P', pi_h, hot_fluid)
    ho_c_init = PropsSI('H', 'P', pi_c, 'T', min(Ti_h, 350.0), cold_fluid) * 1e-5
    sol = scipy.optimize.root(res, np.array([ho_c_init]), method='hybr', tol=1e-5)

    if sol.success:
        ho_c = sol.x[0] * 1e5
        h_h = np.zeros(N)
        h_c = np.zeros(N)
        h_h[0] = hi_h
        h_c[0] = ho_c
        for i in range(N - 1):
            Th = PropsSI('T', 'H', h_h[i], 'P', pi_h, hot_fluid)
            Tc = PropsSI('T', 'H', h_c[i], 'P', pi_c, cold_fluid)
            h_h[i + 1] = h_h[i] - k * A / N * (Th - Tc) / m_h
            h_c[i + 1] = h_c[i] - k * A / N * (Th - Tc) / m_c
    else:
        raise Exception(sol.message)

    po_h = pi_h
    ho_h = h_h[-1]
    mo_h = m_h

    po_c = pi_c
    ho_c = h_c[0]
    mo_c = m_c

    cond.ports["out_r"].p.set_value(po_h)
    cond.ports["out_r"].h.set_value(ho_h)
    cond.ports["out_r"].m.set_value(mo_h)

    cond.ports["out_af"].p.set_value(po_c)
    cond.ports["out_af"].h.set_value(ho_c)
    cond.ports["out_af"].m.set_value(mo_c)

# Expansion valve model
def exp_model(exp):

    Cf = exp.parameter['Cf'].value
    Av = exp.parameter['Av'].value
    hi = exp.ports["in"].h.value
    pi = exp.ports["in"].p.value
    po = exp.ports["out"].p.value
    fluid = exp.ports["in"].fluid

    rho = PropsSI('D', 'P', pi, 'H', hi, fluid)
    pressure_difference = (pi - po) * 1e-5
    n = pressure_difference * 1000 * rho
    m = Cf * Av * np.sqrt(n) / 3600
    ho = hi
    mi = mo = m

    exp.ports["in"].m.set_value(mi)
    exp.ports["out"].h.set_value(ho)
    exp.ports["out"].m.set_value(mo)

# Receiver model
def rev_model(rev):

    p_in = rev.ports["in"].p.value
    m_in = rev.ports["in"].m.value
    fluid = rev.ports["in"].fluid
    p_out = p_in
    h_out = PropsSI('H', 'P', p_in, 'Q', 0.0, fluid)
    m_out = m_in
    rev.ports["out"].p.set_value(p_out)
    rev.ports["out"].h.set_value(h_out)
    rev.ports["out"].m.set_value(m_out)

# Internal heat exchanger model
def ihx_model(ihx):

    k = 1000

    A = ihx.parameter['A'].value

    hi_h = ihx.ports["in_hot"].h.value - 1e-3
    pi_h = ihx.ports["in_hot"].p.value
    mi_h = ihx.ports["in_hot"].m.value
    hot_fluid = ihx.ports["in_hot"].fluid

    hi_c = ihx.ports["in_cold"].h.value + 1e-3
    pi_c = ihx.ports["in_cold"].p.value
    mi_c = ihx.ports["in_cold"].m.value
    cold_fluid = ihx.ports["in_cold"].fluid

    C_h = mi_h * PropsSI('C', 'P', pi_h, 'H', hi_h, hot_fluid)
    C_c = mi_c * PropsSI('C', 'P', pi_c, 'H', hi_c, cold_fluid)
    T_in_h = PropsSI('T', 'P', pi_h, 'H', hi_h, hot_fluid)
    T_in_c = PropsSI('T', 'P', pi_c, 'H', hi_c, cold_fluid)

    R = C_h / C_c
    NTU = k * A / C_h
    n = (-1) * NTU * (1 + R)
    epsilon = (1 - np.exp(n)) / (1 + R)
    Q = epsilon * C_h * (T_in_h - T_in_c)

    po_c = pi_c
    po_h = pi_h
    ho_h = (hi_h - Q / mi_h)
    ho_c = (hi_c + Q / mi_c)
    mo_h = mi_h
    mo_c = mi_c

    ihx.ports["out_hot"].p.set_value(po_h)
    ihx.ports["out_hot"].h.set_value(ho_h)
    ihx.ports["out_hot"].m.set_value(mo_h)

    ihx.ports["out_cold"].p.set_value(po_c)
    ihx.ports["out_cold"].h.set_value(ho_c)
    ihx.ports["out_cold"].m.set_value(mo_c)

# sets the model to each component
net.set_component_model("Comp", comp_model)
net.set_component_model("Cond", cond_model)
net.set_component_model("Evap", evap_model)
net.set_component_model("Exp", exp_model)
net.set_component_model("IHX", ihx_model)
net.set_component_model("Rev", rev_model)

## Network Topology

Connections define the topology only.

In [21]:
net.connect("IHX", "out_cold", "Comp", "in",  fluid_loop=1)
net.connect("Comp", "out", "Cond", "in_r",   fluid_loop=1)
net.connect("Cond", "out_r", "Rev", "in", fluid_loop=1)
net.connect("Rev", "out", "IHX", "in_hot", fluid_loop=1)
net.connect("IHX", "out_hot", "Exp", "in", fluid_loop=1)
net.connect("Exp", "out", "Evap", "in_r", fluid_loop=1)
net.connect("Evap", "out_r", "IHX", "in_cold", fluid_loop=1)


## Parameters and Outputs of the Netwrok

....


In [22]:
net.add_parameter("Comp", "RPM")
net.add_parameter("Cond", "A")
net.add_parameter("Evap", "A")
net.add_parameter("IHX", "A")
net.add_parameter("Exp", "Cf")
net.add_parameter("Exp", "Av")

net.add_output(comp_label='Evap', output_label='Q')


## Constraint Equations

...

In [23]:
def subcooling_fun(net):
    h_s = PropsSI('H', 'P', net.components['Cond'].ports["out_r"].p.value,
                  'Q', 0.0, net.components['Cond'].ports["out_r"].fluid)
    res = (net.components['Cond'].ports["out_r"].h.value - h_s)
    return res

net.add_constraint('subcool_eq', subcooling_fun, ctype='eq', scale_factor=1e-5)

## Loop Breaking

One mass balance equation is removed from the closed refrigerant loop
to avoid linear dependency.


In [24]:
net.add_loop_breaker(fluid_loop=1, junction_id=1)

## Initialization and assigning initial values and bounds to tearing variables

.....

In [25]:
# initialization of the network
net.initialize()

After network initialization all the tearing variables and residual equations of the network are identified by the graph-based tearing algorithm.

The tearing variables can be printed to the console:

In [26]:
# prints tearing variables to the console
net.print_tearing_variables()

Tearing variables:
p(in) at component <Comp> , port <in>
h(in) at component <Comp> , port <in>
p(out) at component <Comp> , port <out>
p(in) at component <Exp> , port <in>
h(in) at component <Exp> , port <in>
p(out) at component <Exp> , port <out>


In this case the tearing variables are:

- Inlet port pressure of the compressor: $p_{i,comp}$
- Inlet port enthalpy of the compressor: $h_{i,comp}$
- Outlet port pressure of the compressor: $p_{o,comp}$
- Inlet port pressure of the expansion valve: $p_{i,exp}$
- Inlet port enthalpy of the compressor: $h_{i,exp}$
- Outlet port pressure of the compressor: $p_{o,exp}$

Which we can write as a vector:

$\mathbf{z} = \begin{bmatrix} p_{i,comp} & h_{i,comp} & p_{o,comp} & p_{i,exp} & h_{i,exp} & p_{o,exp} \end{bmatrix}^{\top}$

Now one can set the initial values as well as the bounds to the tearing variables:

In [27]:
# x_init = [2e5, 4e5, 1e6, 1e6, 2.5e5, 2e5]
x_init = [1.6e5, 4.1e5, 7.9e5, 7.9e5, 2.2e5, 1.6e5]
x_bnds = [(1e5, 4e5), (2e5, 6e5), (5e5, 30e5), (5e5, 30e5), (1e5, 3.5e5), (1e5, 4e5)]

net.set_inital_values(x_init, x_bnds)

We also can print the residual equations to the console:

In [28]:
net.print_residual_equations()

Residual equations:
Pressure Equality at junction 1
Enthalpy Equality at junction 1
Pressure Equality at junction 5
Enthalpy Equality at junction 5
Mass Flow Balance at junction 5


The residual equations of the network are therefore given by:

$r\left(\mathbf{z} \;|\; \mathbf{u} \right) = \mathbf{0} \Leftrightarrow
\left\{
\begin{aligned}
    p_{i,co} - p_{o,ihx,c} &= 0, \\
    h_{i,co} - h_{o,ihx,c} &= 0, \\
    p_{i,exp} - p_{o,ihx,h} &= 0, \\
    h_{i,exp} - h_{o,rihx,h} &= 0, \\
    \dot{m}_{i,exp} - \dot{m}_{o,ihx,h} &= 0,
\end{aligned}
\right.$

where it is indicated that the residuals always depends on the inputs $\mathbf{u}$.

We see that we have 6 unknown tearing variables but only 5 residual equations. Thats due to the fact that we need to add a loop breaker to the closed loop because of the overdetermined mass balance equations.

Therefore, we need to add one constraint equation to close the system. Since the receiver outlet enthalpy, $h_{o,rev}$, depends only on the pressure level within the receiver, $p_{rev}$, energy conservation is not automatically satisfied unless the inlet and outlet specific enthalpies both equal the specific enthalpy of saturated liquid at the receiver pressure:

$h_{o,rev} = h_{i,rev} = h_s(p_{rev})$.

From the energy balance at the junction between the condenser outlet and the receiver inlet, we have:

$h_{i,rev} = h_{o,cond}$.

This implies that the condenser outlet enthalpy must also match the saturation enthalpy at the receiver pressure to ensure consistency and prevent energy imbalances in steady-state operation.

Thus, the following constraint equation is formulated:

$h_{o,r,cond} - h_{s}\left(p_{o,cond}\right) = 0$.

We can easily add such a constraint within the framework the following:


In [ ]:
# define constraint as a callable python function
def sat_con(net):
    h_s = PropsSI('H', 'P', net.components['Cond'].ports["out_r"].p.value,
                  'Q', 0.0, net.components['Cond'].ports["out_r"].fluid)
    res = (net.components['Cond'].ports["out_r"].h.value - h_s)
    return res

# adds constraint to the network
net.add_constraint("saturation_constraint", sat_con, ctype='eq', scale_factor=1e-5)

## Direct problem formulation

First lets consider the direct problem. In this case we want to simulate all the thermal-hydraulic state $\mathbf{x}$ and the outputs of the system which includes the heat loads of the evaporator $\dot{Q}_{evap}$ and condenser $\dot{Q}_{cond}$ as well as the power consumption of the compressor $P_{comp}$ given the inputs of the system, which are the antifrogen inlet states at the evaporator and condenser.

In that case the input vector is given by:

$\mathbf{u} = \begin{bmatrix}\omega & A_{v} & p_{i,af,evap} & h_{i,af,evap} & \dot{m}_{i,af,evap} & p_{i,af,cond} & h_{i,af,cond} & \dot{m}_{i,af,cond} & \end{bmatrix}^{\top}$,

and the output vector

$\mathbf{y} = \begin{bmatrix}\dot{Q}_{evap} & \dot{Q}_{cond} & P_{comp} \end{bmatrix}^{\top}$.


The direct problem is then mathematically stated as:

Given the inputs $\mathbf{u}$, we solve for the tearing variables $\mathbf{z}$ by solving the system:

$r\left(\mathbf{z} \;|\; \mathbf{u} \right) = \mathbf{0} \Leftrightarrow
\left\{
\begin{aligned}
    p_{i,co} - p_{o,ihx,c} &= 0, \\
    h_{i,co} - h_{o,ihx,c} &= 0, \\
    p_{i,exp} - p_{o,ihx,h} &= 0, \\
    h_{i,exp} - h_{o,rihx,h} &= 0, \\
    \dot{m}_{i,exp} - \dot{m}_{o,ihx,h} &= 0.
\end{aligned}
\right.$

For which then with the mapping

Given some system inputs $\mathbf{u}$, the objective is to find a solution of tearing variables $\mathbf{z}$ such that $\mathbf{r}(\mathbf{z} \;|\; \mathbf{u})=\mathbf{0}$, which then allows for the calculation of the corresponding internal system variables $\mathbf{x}$ and system outputs $\mathbf{y}$ by the mapping

$\[\left[\begin{matrix}
		\mathbf{x}  \\
		\mathbf{y} 
\end{matrix} \right] = \mathbf{\psi}\left(\mathbf{z^*}, \mathbf{u}\right),
\]$

where $\mathbf{z^*}$ is a specific solution of $\mathbf{r}(\mathbf{z} \;|\; \mathbf{u})=\mathbf{0}$.

In other words one wants to find all the pressure, specific enthalpy and mass flow rate values at each inlet and outlet port of the components, as well as all system outputs, such that steady-state mass and energy balance and pressure equality equations are satisfied at each junction, given some system inputs. 

In this case the physical and computational causality are the same and therefore it is a direct problem.

### Boundary Conditions

To achieve this in the framework, first we need to assign some boundary conditions to the system.

In this case there are two ports for which we need to assign boundary condition values. Because of the causality structure of the defined components.
 
We see that there are ports which are not connected to a junction but the variables of these ports are inputs of the components. Therefore we need to assign the boundary condition values of the following variables:
- Antifrogen inlet port pressure, enthalpy and mass flow rate at the evaporator: $p_{i,af,evap}$, $h_{i,af,evap}$, $\dot{m}_{i,af,evap}$
- Antifrogen inlet port pressure, enthalpy and mass flow rate at the condenser: $p_{i,af,cond}$, $h_{i,af,cond}$, $\dot{m}_{i,af,cond}$

Therefore we assign the boundary conditions to these variables:

In [29]:
# assigning boundary conditions to the evaporator inlet port states
hi_evap_af = PropsSI("H", "P", 1e5, "T", -5.0+273.15, net.fluid_loops[2])
net.set_bc("Evap", "in_af", "p", 1e5, fluid_loop=2)
net.set_bc("Evap", "in_af", "h", hi_evap_af, fluid_loop=2)
net.set_bc("Evap", "in_af", "m", 1.0, fluid_loop=2)

# assigning boundary conditions to the condenser inlet port states
hi_cond_af = PropsSI("H", "P", 1e5, "T", 27.0+273.15, net.fluid_loops[3])
net.set_bc("Cond", "in_af", "p", 1e5, fluid_loop=3)
net.set_bc("Cond", "in_af", "h", hi_cond_af, fluid_loop=3)
net.set_bc("Cond", "in_af", "m", 1.0, fluid_loop=3)

## Parameter Setting



In [30]:

net.set_parameter("Comp", "RPM", value=2500)
net.set_parameter("Exp", "Av", value=0.3)
net.set_parameter("Exp", "Cf", value=0.25)
net.set_parameter("Cond", "A", value=3.65)
net.set_parameter("Evap", "A", value=2.56)
net.set_parameter("IHX", "A", value=0.75)


In [31]:
net.solve_system_notebook()


 Start solver...
Residual norm = 1.408e+00, Obj = 1.458e+01
Residual norm = 1.520e-01, Obj = 4.262e+00
Residual norm = 4.949e-02, Obj = 8.012e-01
Residual norm = 5.152e-01, Obj = 9.945e+00
Residual norm = 8.123e-03, Obj = 4.150e-01
Residual norm = 6.302e-03, Obj = 2.156e-01
Residual norm = 3.288e-01, Obj = 4.192e+00
Residual norm = 2.138e-01, Obj = 2.257e+00
Residual norm = 1.705e+00, Obj = 1.479e+01
Residual norm = 1.518e+00, Obj = 1.778e+01
Residual norm = 1.406e-04, Obj = 8.751e-03
Residual norm = 2.564e-06, Obj = 3.682e-03
Residual norm = 1.285e-03, Obj = 2.686e-02
Residual norm = 2.377e-02, Obj = 3.785e-01
Residual norm = 2.247e-08, Obj = 9.270e-04
Residual norm = 2.086e-08, Obj = 5.737e-04
Residual norm = 4.431e-11, Obj = 1.618e-05
Residual norm = 2.167e-12, Obj = 6.012e-06
Residual norm = 5.128e-09, Obj = 9.318e-04
Residual norm = 1.263e-07, Obj = 4.657e-03
Residual norm = 8.554e-13, Obj = 2.222e-07
Optimization terminated successfully    (Exit mode 0)
            Current funct

In [32]:
res = net.get_results()

pi_comp = res["ports"]["Comp"]["in"]["p"]
po_comp = res["ports"]["Comp"]["out"]["p"]
hi_comp = res["ports"]["Comp"]["in"]["h"]
ho_comp = res["ports"]["Comp"]["out"]["h"]
mi_comp = res["ports"]["Comp"]["in"]["m"]
mo_comp = res["ports"]["Comp"]["out"]["m"]



## Inverse Problem

...

In [ ]:
net.set_parameter("Comp", "RPM", value=2500, is_var=True, scale_factor=1.0e-3, bounds=(750, 3500))
net.set_parameter("Exp", "Av", value=0.3, is_var=True, scale_factor=1.0, bounds=(0.1, 1.0))


def superheat_fun(net):
    SH_value = 5.0
    T_sat = PropsSI('T', 'P', net.components['Evap'].ports["out_r"].p.value,
                    'Q', 1.0, net.components['Evap'].ports["out_r"].fluid)
    h_SH = PropsSI('H', 'T', T_sat + SH_value,
                   'P', net.components['Evap'].ports["out_r"].p.value,
                   net.components['Evap'].ports["out_r"].fluid)
    res = np.abs(net.components['Evap'].ports["out_r"].h.value - h_SH)
    return res

def cooling_load(net):
    Qsp = 18000
    Q = net.components['Evap'].outputs['Q'].value
    res = np.abs(Q - Qsp)
    return res

net.add_constraint('supheat_eq', superheat_fun, ctype='obj', weight_factor=1e-5)
net.add_constraint('cool_load_eq', cooling_load, ctype='obj', weight_factor=1e-3)


net.solve_system_notebook()
